In [2]:
import pytorch_lightning as pl
from models.inpainting_model import InpaintingModel
from models.flow_model import FlowModel
from models.occlusion_model import OcclusionModel
from models.flow_occ_model import FlowOccModel
from models.networks.lightning_datamodule import ImageFlowOccModule
from models.data.utils.flow_utils import flow2img
import matplotlib.pyplot as plt
import torch
# flow occ model should be ['simple', 'flowoccnets', 'flowoccnetc', 'pwoc', 'flowoccnet', 'eflownet']
# optical flow model should be ['simple', 'flownets', 'flownetc', 'pwc', 'flownet']
# occlusion model should be ['simple', 'occnets', 'occnetc']
hparams = dict(learning_rate=1e-3, model='simple')
root = r'C:\Users\caodo\Desktop\TUM\data'
model = FlowModel(root=root, hparams=hparams)


In [3]:
data_module = ImageFlowOccModule(root = root, image_size= (128,256), batch_size=1)
data_module.prepare_data()
data_module.setup()

In [ ]:
dataloader =data_module.train_dataloader()
img, flow, occ = next(iter(dataloader))

In [ ]:
img1 = img[0,:3,:,:]
img2 = img[0,3:,:,:]
flow_ = flow[0,:,:,:]
occ_ = occ[0,:,:,:]

In [ ]:
img1 = img1.detach().cpu().numpy().transpose(1, 2, 0)
img2 = img2.detach().cpu().numpy().transpose(1, 2, 0)
occ_ = occ_.detach().cpu().numpy().transpose(1, 2, 0)
flow_ = flow_.detach().cpu().numpy().transpose(1, 2, 0)

In [ ]:
im_flow = flow2img(flow_)

In [ ]:

plt.figure(figsize=(12, 4 * 5))
plt.subplot(4, 1, 1)
plt.imshow(img1/2.0 + 0.5)
plt.title('image 1')

plt.subplot(4, 1, 2)
plt.imshow(img2/2.0 + 0.5)
plt.title('image 2')

plt.subplot(4, 1, 3)
plt.imshow(occ_)
plt.title('occlusion map')

plt.subplot(4, 1, 4)
plt.imshow(im_flow)
plt.title('optical flow')

In [ ]:

flow_bt = model(torch.unsqueeze(img[0],0))
flow_bt = flow_bt.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
im_flow_bt = flow2img(flow_bt)
plt.imshow(im_flow_bt)
plt.title('optical flow')

In [4]:
max_epochs = 100

trainer = pl.Trainer(
    max_epochs=max_epochs, gpus=1, overfit_batches = 1
)

trainer.fit(model, data_module)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params
----------------------------------------
0 | model | SimpleFlowNet | 232 K 
C:\Users\caodo\anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: You requested to overfit but enabled test/val dataloader shuffling. We are turning it off for you.
  warnings.warn(*args, **kwargs)


C:\Users\caodo\anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: You requested to overfit but enabled training dataloader shuffling. We are turning it off for you.
  warnings.warn(*args, **kwargs)


C:\Users\caodo\anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [ ]:
#model.load_from_checkpoint(checkpoint_path='./lightning_logs/lr=4e-3,whole dataset/checkpoints/epoch=32.ckpt')
flow_at = model(torch.unsqueeze(img[0],0))
flow_at = flow_at.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
im_flow_at = flow2img(flow_at)
plt.imshow(im_flow_at)
plt.title('optical flow after training ')

In [1]:
from models.networks.flow_net import FlowNet
from models.networks.efficient_flow_net import EFlowNet, EFlowNet2
from models.networks.flow_net_s import FlowNetS
from models.networks.cost_volume_flow_net import FlowNetCV
from models.networks.simple_flow_net import SimpleFlowNet
from torchsummary import summary

In [2]:
net = SimpleFlowNet().cuda() 
summary(net, [6, 64, 128])

Layer (type:depth-idx)                   Output Shape              Param #
├─Downsample: 1-1                        [-1, 16, 32, 64]          --
|    └─Conv2d: 2-1                       [-1, 6, 32, 64]           144
|    └─BatchNorm2d: 2-2                  [-1, 6, 32, 64]           12
|    └─LeakyReLU: 2-3                    [-1, 6, 32, 64]           --
|    └─Conv2d: 2-4                       [-1, 6, 32, 64]           324
|    └─BatchNorm2d: 2-5                  [-1, 6, 32, 64]           12
|    └─LeakyReLU: 2-6                    [-1, 6, 32, 64]           --
|    └─Conv2d: 2-7                       [-1, 16, 32, 64]          96
|    └─BatchNorm2d: 2-8                  [-1, 16, 32, 64]          32
|    └─LeakyReLU: 2-9                    [-1, 16, 32, 64]          --
├─Downsample: 1-2                        [-1, 32, 16, 32]          --
|    └─Conv2d: 2-10                      [-1, 8, 16, 32]           512
|    └─BatchNorm2d: 2-11                 [-1, 8, 16, 32]           16
|    └─Leaky

Layer (type:depth-idx)                   Output Shape              Param #
├─Downsample: 1-1                        [-1, 16, 32, 64]          --
|    └─Conv2d: 2-1                       [-1, 6, 32, 64]           144
|    └─BatchNorm2d: 2-2                  [-1, 6, 32, 64]           12
|    └─LeakyReLU: 2-3                    [-1, 6, 32, 64]           --
|    └─Conv2d: 2-4                       [-1, 6, 32, 64]           324
|    └─BatchNorm2d: 2-5                  [-1, 6, 32, 64]           12
|    └─LeakyReLU: 2-6                    [-1, 6, 32, 64]           --
|    └─Conv2d: 2-7                       [-1, 16, 32, 64]          96
|    └─BatchNorm2d: 2-8                  [-1, 16, 32, 64]          32
|    └─LeakyReLU: 2-9                    [-1, 16, 32, 64]          --
├─Downsample: 1-2                        [-1, 32, 16, 32]          --
|    └─Conv2d: 2-10                      [-1, 8, 16, 32]           512
|    └─BatchNorm2d: 2-11                 [-1, 8, 16, 32]           16
|    └─Leaky

In [3]:
net = EFlowNet2().cuda() 
summary(net, [6, 64, 128])

Layer (type:depth-idx)                   Output Shape              Param #
├─InitialBlock: 1-1                      [-1, 16, 32, 64]          --
|    └─MaxPool2d: 2-1                    [-1, 6, 32, 64]           --
|    └─Conv2d: 2-2                       [-1, 10, 32, 64]          550
|    └─BatchNorm2d: 2-3                  [-1, 16, 32, 64]          32
|    └─PReLU: 2-4                        [-1, 16, 32, 64]          16
├─BottleNeck: 1-2                        [-1, 64, 16, 32]          --
|    └─MaxPool2d: 2-5                    [-1, 16, 16, 32]          --
|    └─Conv2d: 2-6                       [-1, 4, 16, 32]           256
|    └─BatchNorm2d: 2-7                  [-1, 4, 16, 32]           8
|    └─PReLU: 2-8                        [-1, 4, 16, 32]           1
|    └─Conv2d: 2-9                       [-1, 4, 16, 32]           144
|    └─BatchNorm2d: 2-10                 [-1, 4, 16, 32]           8
|    └─PReLU: 2-11                       [-1, 4, 16, 32]           1
|    └─Conv2d: 2

Layer (type:depth-idx)                   Output Shape              Param #
├─InitialBlock: 1-1                      [-1, 16, 32, 64]          --
|    └─MaxPool2d: 2-1                    [-1, 6, 32, 64]           --
|    └─Conv2d: 2-2                       [-1, 10, 32, 64]          550
|    └─BatchNorm2d: 2-3                  [-1, 16, 32, 64]          32
|    └─PReLU: 2-4                        [-1, 16, 32, 64]          16
├─BottleNeck: 1-2                        [-1, 64, 16, 32]          --
|    └─MaxPool2d: 2-5                    [-1, 16, 16, 32]          --
|    └─Conv2d: 2-6                       [-1, 4, 16, 32]           256
|    └─BatchNorm2d: 2-7                  [-1, 4, 16, 32]           8
|    └─PReLU: 2-8                        [-1, 4, 16, 32]           1
|    └─Conv2d: 2-9                       [-1, 4, 16, 32]           144
|    └─BatchNorm2d: 2-10                 [-1, 4, 16, 32]           8
|    └─PReLU: 2-11                       [-1, 4, 16, 32]           1
|    └─Conv2d: 2